In [9]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os.path
import pickle
import pandas as pd
import yaml
from rdflib import Graph, URIRef, Literal, BNode, Namespace
from rdflib.namespace import DC, RDFS, FOAF, DCTERMS, VOID, RDF, XSD, OWL
import requests
import numpy as np

# Cleaning the spreadsheet

This notebook won't be necessary once the final spreadsheet is prepared as a supplementary material.

In [10]:
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f.read())
    SCOPES = config['SCOPES']
    SPREADSHEET_ID = config['SPREADSHEET']
    SHEET = config['SHEET']

In [11]:
def get_google_sheet(sheet, spreadsheet_id=SPREADSHEET_ID):
    creds = None

    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server()
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    service = build('sheets', 'v4', credentials=creds)
    gsheet = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=sheet).execute()
    values = gsheet.get('values', [])
    if not values:
        print('No data found.')
    else:
        df = pd.DataFrame(values, columns=values[0]).tail(-1)
        return df

In [12]:
data = get_google_sheet(SHEET)
dd = data

In [13]:
data.sample(4)

,reference_id,material_description,source,material,erm,hash,material_type,placeholder,material_iri,material_synthesis_characterization_quote,...,pah_impurities_instrument,endotoxin_impurities,endotoxin_impurities_unit,endotoxin_impurities_instrument,endotoxins_impurities_instrument,crystal_structure,crystal_structure_instrument,polidispersity_index_(%),doping,polidispersity_index_instrument
78,13,SiNP (50nm),https://pubmed.ncbi.nlm.nih.gov/26069526,SiNP1,ERM:857785,857785,Si,http://purl.enanomapper.org/onto/,http://purl.bioontology.org/ontology/npo#NPO_1373,,...,None,None,None,None,None,None,None,None,None,None
272,41,,unspecific,,,,,,,,...,None,None,None,None,None,None,None,None,None,None
138,21,GO,https://doi.org/10.3390%2Fnano9081180,GO21,ERM:192679,192679,GO,http://purl.enanomapper.org/onto/,http://purl.enanomapper.org/onto/ENM_0000117,,...,None,None,None,None,None,None,None,None,None,None
385,41,,unspecific,,,,,,,,...,None,None,None,None,None,None,None,None,None,None


In [14]:
data.head(2)

,reference_id,material_description,source,material,erm,hash,material_type,placeholder,material_iri,material_synthesis_characterization_quote,...,pah_impurities_instrument,endotoxin_impurities,endotoxin_impurities_unit,endotoxin_impurities_instrument,endotoxins_impurities_instrument,crystal_structure,crystal_structure_instrument,polidispersity_index_(%),doping,polidispersity_index_instrument
1,1,ZnO-NPs and TiO2 particles were obtained from ...,https://doi.org/10.1016/j.taap.2010.04.012,ZnO2,ERM:789399,789399,ZnO,http://purl.enanomapper.org/onto/,http://purl.bioontology.org/ontology/npo#NPO_1542,,...,None,None,None,None,None,None,None,None,None,None
2,1,ZnO-NPs and TiO2 particles were obtained from ...,https://doi.org/10.1016/j.taap.2010.04.012,ZnO2,ERM:789399,789399,ZnO,http://purl.enanomapper.org/onto/,http://purl.bioontology.org/ontology/npo#NPO_1542,,...,,below detection limit,,diazo-coupling Limulus Amebocyte Lysate (LAL),None,None,None,None,None,None


In [15]:
# Remove rows based on specific conditions
data = data[data['source'] != 'unspecific']
data = data[data['source'] != '']
data = data[data['s'] != ""]
data = data[data['o'] != ""]
data = data[data['s_iri'] != ""]
data = data[data['o_iri'] != '']

# Replace individual whitespace cells with '#N/A'
data.replace(r'^\s*$', '#N/A', regex=True, inplace=True)

# Replace entire rows that are whitespace with '#N/A'
data.loc[data.apply(lambda x: x.str.isspace().all(), axis=1)] = '#N/A'

# Replace empty strings with np.nan
data.replace('', np.nan, inplace=True)

In [16]:
nodes = [i for i in data.columns if "_iri" in i]
nodes

['material_iri',
 's_iri',
 'o_iri',
 'model_cell_iri',
 'organism_iri',
 'exposure_time_units_iri',
 'dose_units_iri',
 'endpoint_iri',
 'assay_iri',
 'primary_size_descriptor_iri',
 'hydrodymanic_diameter_method_iri']

In [17]:
units = [i for i in data.columns if "_units" in i]
units

['exposure_time_units',
 'exposure_time_units_iri',
 'dose_units',
 'dose_units_iri',
 'endpoint_units']

In [18]:
quotes = [i for i in data.columns if "quote" in i]
quotes

['material_synthesis_characterization_quote',
 'quote',
 'primary_size_quote',
 'bundle_diameter_quote']

In [19]:
qualifiers = [i for i in data.columns if "qualifier" in i]
qualifiers

['qualifier',
 'qualifier_value',
 'primary_size_qualifier',
 'primary_size_qualifier_value',
 'bundle_diameter_qualifier',
 'bundle_diameter_qualifier_value',
 'nominal_size_qualifier',
 'nominal_size_qualifier_value',
 'average_grain_size_qualifier',
 'average_grain_size_qualifier_value',
 'agglomerate_size_qualifier',
 'agglomerate_size_qualifier_value',
 'aggregate_size_qualifier',
 'aggregate_size_qualifier_value',
 'diameter_qualifier',
 'diameter_qualifier_value',
 'hydrodynamic_diameter_qualifier',
 'hydrodynamic_diameter_qualifier_value',
 'surface_area_qualifier',
 'surface_area_qualifier_value',
 'zeta_potential_qualifier',
 'zeta_potential_qualifier_value',
 'porosity_qualifier',
 'porosity_qualifier_value']

In [20]:
others = [i for i in data.columns if i not in quotes and i not in units and i not in nodes and i not in qualifiers]
others

['reference_id',
 'material_description',
 'source',
 'material',
 'erm',
 'hash',
 'material_type',
 'placeholder',
 's',
 'p',
 'o',
 'uses_assay_type',
 'original_study',
 'citesAsSourceDocument',
 'model_cell',
 'organism',
 'organism_age',
 'manufacturer_cell',
 'exposure_time',
 'dose',
 'exposure_route',
 'endpoint',
 'endpoint_measurement',
 'assay_equipment',
 'assay',
 'primary_size_descriptor__(ecd,_feret_diameter_,_length)',
 'primary_size',
 'primary_size_unit',
 'normalized_primary_size',
 'primary_size_method',
 'primary_size_instrument',
 'bundle_diameter',
 'bundle_diameter_unit',
 'normalized_bundle_diameter',
 'bundle_diameter_method',
 'bundle_diameter_instrument',
 'nominal_size',
 'nominal_size_unit',
 'average_grain_size',
 'average_grain_size_unit',
 'agglomerate_size',
 'agglomerate_size_unit',
 'agglomerate_size_method',
 'aggregate_size',
 'aggregate_size_unit',
 'aggregate_size_medium',
 'aggregate_size_instrument',
 'aggregate_size_method',
 'diameter_unit'

Save all IRIs

In [21]:
iris_stack = data[nodes].stack()
iris = pd.DataFrame(list(set(iris_stack[iris_stack!="#N/A"])))
# Nanowiki too
nanowiki = pd.read_csv('../data/nanowiki.csv')
# Pattern to match
pattern = 'https:'
nanowiki_stack = nanowiki.stack()

# Use str.contains() to check if the cell value contains the pattern
iris_nanowiki = [i for i in nanowiki_stack if 'http' in str(i)]

iris = pd.concat([iris, pd.DataFrame(iris_nanowiki)])
iris.to_csv("../data/iris.tsv", index=False, header=False)

Save data

In [22]:
data.to_csv('../data/causal_network.csv', index=True, index_label='row_id')